In [1]:
import onnx
import torch
import numpy as np
import pandas as pd  
import copy

from onnx2pytorch import ConvertModel
from torch import nn

from algorithms.iterative_relaxation import IterativeRelaxation
from algorithms.decision_procedure import MarabouCoreDP
from algorithms.decision_tree import DecisionTree

from models.test_models import ProphecyPaperNetwork, TestModel
from models.acasxu_1_1 import Acasxu1_1
from models.utils import attach_relu_activation_hook, attach_layer_output_hook, get_layers_info, turn_bool_activation_to_int


torch.set_printoptions(precision=8) 

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
model = Acasxu1_1()
model.load_state_dict(torch.load('./models/acasxu_1_1.pt'))
_act_handles, activation_signature = attach_relu_activation_hook(model)

In [3]:
model

Acasxu1_1(
  (fc0): Linear(in_features=5, out_features=50, bias=True)
  (relu0): ReLU()
  (fc1): Linear(in_features=50, out_features=50, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=50, out_features=50, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=50, out_features=50, bias=True)
  (relu3): ReLU()
  (fc4): Linear(in_features=50, out_features=50, bias=True)
  (relu4): ReLU()
  (fc5): Linear(in_features=50, out_features=50, bias=True)
  (relu5): ReLU()
  (output_layer): Linear(in_features=50, out_features=5, bias=True)
)

In [4]:
from network_properties.read_vnnlib import read_vnnlib
from pathlib import Path
vnnlib_path = Path("./network_properties/prop_1.vnnlib") # acasxu
# vnnlib_path = Path("./network_properties/prop_0_0.03.vnnlib") # MNIST

input_ranges, specifications = read_vnnlib(vnnlib_path)[0]
print(input_ranges)
print(specifications)

5 inputs and 5 outputs in vnnlib
[[0.6, 0.679857769], [-0.5, 0.5], [-0.5, 0.5], [0.45, 0.5], [-0.5, -0.45]]
[(array([[-1.,  0.,  0.,  0.,  0.]]), array([-3.99112565]))]


## 0. Prepare ACASXU dataset

In [5]:
# !wget https://raw.githubusercontent.com/safednn-nasa/prophecy_DNN/master/clusterinACAS_0_short.csv -O ./datasets/clusterinACAS_0_shrt.csv

In [6]:
acas_train = np.empty([384221,5],dtype=float)
acas_train_labels = np.zeros(384221,dtype=int)

def read_inputs_from_file(inputFile):
  global acas_train, acas_train_labels, num
  with open(inputFile) as f:
    lines = f.readlines()
    print(len(lines), "examples")
    acas_train = np.empty([len(lines),5],dtype=float)
    acas_train_labels = np.zeros(len(lines),dtype=int)

    for l in range(len(lines)):
      # This is to remove the useless 1 at the start of each string. Not sure why that's there.
      k = [float(stringIn) for stringIn in lines[l].split(',')] 
      
      # acas_train[l+num] = np.zeros(5,dtype=float) 
      # we're asuming that everything is 2D for now. The 1 is just to keep numpy happy.
      if len(k) > 5:
        lab = int(k[5])
        #if ((lab == 0) or (lab == 2)):
        #  lab = 0
        #else:
        #  lab = 1
        acas_train_labels[l+num] = lab

      count = 0
      for i in range(0,5):
        #print(count)
        acas_train[l+num][i] = k[i]
        #print(k[i])

In [7]:
num = 0
read_inputs_from_file('./datasets/clusterinACAS_0_shrt.csv')
print(acas_train.shape)
print(acas_train_labels.shape)
acas_train[:5]

384221 examples
(384221, 5)
(384221,)


array([[-0.320142, -0.5     , -0.5     , -0.5     , -0.5     ],
       [-0.295234, -0.5     , -0.5     , -0.5     , -0.5     ],
       [-0.240207, -0.5     , -0.5     , -0.5     , -0.5     ],
       [-0.208943, -0.5     , -0.5     , -0.5     , -0.5     ],
       [-0.183636, -0.5     , -0.5     , -0.5     , -0.5     ]])

In [8]:
def create_df(inputs, predicted_labels, true_labels, activation_signature):
  data = []
  for index, input_data in enumerate(inputs):
    data_point = { 
      "input": input_data, 
      "true_label": true_labels[index], 
      "predicted_label": predicted_labels[index].item(),
    }
    for name, layer_activation in activation_signature.items():
      data_point[name] = layer_activation[index]
    data.append(data_point)

  return pd.DataFrame(data)

outputs = model(torch.tensor(acas_train, dtype=torch.float32))
predicted_labels = torch.argmin(outputs, dim=1)
activation_signature = turn_bool_activation_to_int(activation_signature, to_list=True)
df = create_df(acas_train, predicted_labels, acas_train_labels, activation_signature)
df.head(10)

,input,true_label,predicted_label,relu0,relu1,relu2,relu3,relu4,relu5
0,"[-0.320142, -0.5, -0.5, -0.5, -0.5]",3,3,"[1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, ...","[0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, ...","[0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
1,"[-0.295234, -0.5, -0.5, -0.5, -0.5]",1,1,"[1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, ...","[1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, ...","[0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, ...","[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[-0.240207, -0.5, -0.5, -0.5, -0.5]",0,0,"[1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, ...","[1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[-0.208943, -0.5, -0.5, -0.5, -0.5]",0,0,"[1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, ...","[1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[-0.183636, -0.5, -0.5, -0.5, -0.5]",0,0,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, ...","[1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,"[-0.10156, -0.5, -0.5, -0.5, -0.5]",0,0,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, ...","[1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,"[-0.000732, -0.5, -0.5, -0.5, -0.5]",0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,"[0.100096, -0.5, -0.5, -0.5, -0.5]",0,0,"[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
8,"[0.200941, -0.5, -0.5, -0.5, -0.5]",0,0,"[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9,"[0.301769, -0.5, -0.5, -0.5, -0.5]",0,0,"[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [9]:
# test accuracy
len(df[df["predicted_label"] == df['true_label']]) / (len(df))

0.7663037678835878

## 1. Layer properties

### 1.1 Find candidates with Decision Tree

In [10]:
relu_layers = ['relu0', 'relu1', 'relu2', 'relu3', 'relu4']
labels = [0,1,2,3,4]
# for name, module in list(model.named_modules()):
#   if isinstance(module, torch.nn.ReLU):
#     relu_layers.append(name)
# relu_layers

In [11]:
# for layer_name in relu_layers: 
#   for chosen_label in [0,1,2,3,4]:
#     df['satisfies_postcon'] = np.where(df['predicted_label'] == chosen_label, 1, -1)
#     decision_tree = DecisionTree(df, X_col=layer_name, Y_col="satisfies_postcon")
#     leaves_with_activation_pattern = decision_tree.get_potential_layer_properties()

In [12]:
df['satisfies_postcon'] = np.where(df['predicted_label'] == 0, 1, -1)
decision_tree = DecisionTree(df, X_col='relu4', Y_col="satisfies_postcon")
leaves_with_activation_pattern = decision_tree.get_potential_layer_properties()

num_of_patterns = len(leaves_with_activation_pattern)
print(f"num_of_patterns: {num_of_patterns}")

total_support = sum(candidate['support'] for candidate in leaves_with_activation_pattern)
print(f"total_support: {total_support}")

top_5_supports = [candidate['support'] for candidate in leaves_with_activation_pattern[:5]]
print(f"top_5_supports: {top_5_supports}")

num_of_patterns: 810
total_support: 237734
top_5_supports: [109147, 58070, 9977, 8950, 7424]
